In [0]:
import json
from glob import glob
from functions.sanity import validate_settings, initialize_schemas_and_volumes, initialize_empty_tables, check_host_name_matches_catalog
from functions.utility import apply_job_type

# Load anything in layer_*_<color>/*.json
job_settings = {}
for color in [ 'bronze', 'silver', 'gold' ]:
    paths = glob(f'./layer_*_{color}/*.json')
    job_settings[color] = []
    for path in paths:
        table_name = path.split('/')[-1].split('.')[0]
        with open(path) as f:
            settings = json.load(f)
        settings = apply_job_type(settings)
        history = {
            'build_history': settings.get('build_history', 'false'),
            'history_schema': settings.get('history_schema'),
            'full_table_name': settings.get('dst_table_name'),
        }
        job_settings[color].append({'table': table_name, 'history': history})
for key, value in job_settings.items():
    dbutils.jobs.taskValues.set(key=key, value=value)

# Sanity check
validate_settings(dbutils)
initialize_schemas_and_volumes(spark)
initialize_empty_tables(spark)
check_host_name_matches_catalog(dbutils)
